Read in parameter values from  [``/parameters/calibration_zeta_0.75.csv``](/parameters/calibration_zeta_0.75.csv) which reads in the values for 0.75 zeta

Read in parameter values from  [``/parameters/calibration_zeta_1.25.csv``](/parameters/calibration_zeta_1.25.csv) which reads in the values for 1.25 zeta

Read in baseline parameter values from  [``/parameters/calibration_params.csv``](/parameters/calibration_params.csv)

## Setup

In [22]:
using PerlaTonettiWaugh, Parameters, Suppressor

In [23]:
# generate params
params = parameter_defaults()
d = params.d_0

ζ_075 = merge(params, (ζ = 0.75, κ = 0.078147243594978, d = d))
ζ_1 = merge(params, (d = params.d_0,)) # baseline
ζ_125 = merge(params, (ζ = 1.25, κ = 0.130245405991629, d = d));

d_lowered = 1 + 0.90*(d-1)

ζ_075_lowered = merge(ζ_075, (d = d_lowered,))
ζ_1_lowered = merge(ζ_1, (d = d_lowered,)) # baseline
ζ_125_lowered = merge(ζ_125, (d = d_lowered,));

## Zeta 1

### Baseline Allocations

In [24]:
sol_1 = stationary_algebraic(ζ_1, settings_defaults())
for key in (:g, :Ω, :z_hat, :λ_ii, :c)
    val = sol_1[key]
    println("$(key) = $(val)")
end

g = 0.007913401963163525
Ω = 0.6805904609927736
z_hat = 1.9813268967445292
λ_ii = 0.893708728294921
c = 0.8706834939103543


### Counterfactual (Lowered) Allocations and Consumption Equivalent

In [25]:
sol_1_lowered = stationary_algebraic(ζ_1_lowered, settings_defaults())
for key in (:g, :Ω, :z_hat, :λ_ii, :c)
    val = sol_1_lowered[key]
    println("$(key) = $(val)")
end

g = 0.010250822794505147
Ω = 0.6459867736680407
z_hat = 1.8456817342356466
λ_ii = 0.8555713586729671
c = 0.862941967203508


In [26]:
ce_1 = 100*consumption_equivalent(sol_1_lowered.U_bar, sol_1.U_bar, ζ_1)

11.181899506570137

### Transition Dynamics and Consumption Equivalent

In [27]:
overall_parameters = merge(ζ_1, (d_0 = d, d_T = d_lowered,));
settings = settings_defaults()
@unpack stationary_T, stationary_0, change_welfare, change_trade = compare_steady_states(overall_parameters, settings);

In [28]:
abs(stationary_T.g - sol_1_lowered.g) < 1e-5 && abs(stationary_0.g - sol_1.g) < 1e-5 # check that the grid is close enough

true

In [ ]:
cachename = model_cachename(overall_parameters, settings) # uniquely identify the run by parameters + settings
cachepath = joinpath(pwd(), "data", cachename * ".csv")
use_cache = true # change this to false if you don't want to use the cache for a run

if isfile(cachepath) && use_cache # read if possible
    println("using cache.")
    solved = CSV.read(cachepath);
else
    println("solving without cache.")
    @time solution = @suppress solve_transition(overall_parameters, settings);
    solved = solution.results;
end

In [30]:
filter!(row -> row.t >= 0, solved) # don't plot pre-shock values
@show ce_1_transition = 100*consumption_equivalent(solved.U[1], stationary_0.U_bar, overall_parameters);

ce_1_transition = 100 * consumption_equivalent(solved.U[1], stationary_0.U_bar, overall_parameters) = 10.802117356508889


## Zeta 0.75

### Baseline Allocations

In [31]:
sol_075 = stationary_algebraic(ζ_075, settings_defaults())
for key in (:g, :Ω, :z_hat, :λ_ii, :c)
    val = sol_075[key]
    println("$(key) = $(val)")
end

g = 0.007913401963163787
Ω = 0.9074539479905924
z_hat = 1.9813268967445021
λ_ii = 0.8937087282949173
c = 0.9943001206514334


### Counterfactual (Lowered) Allocations and Consumption Equivalent

In [32]:
sol_075_lowered = stationary_algebraic(ζ_075_lowered, settings_defaults())
for key in (:g, :Ω, :z_hat, :λ_ii, :c)
    val = sol_075_lowered[key]
    println("$(key) = $(val)")
end

g = 0.010250822798546106
Ω = 0.8613156829790775
z_hat = 1.845681734222669
λ_ii = 0.855571358670515
c = 0.985459476381832


In [33]:
ce_075 = 100*consumption_equivalent(sol_075_lowered.U_bar, sol_075.U_bar, ζ_075)

11.181899256228167

### Transition Dynamics and Consumption Equivalent

In [34]:
overall_parameters = merge(ζ_075, (d_0 = d, d_T = d_lowered,));
settings = settings_defaults()
@unpack stationary_T, stationary_0, change_welfare, change_trade = compare_steady_states(overall_parameters, settings);

In [35]:
abs(stationary_T.g - sol_075_lowered.g) < 1e-5 && abs(stationary_0.g - sol_075.g) < 1e-5 # check that the grid is close enough

true

In [36]:
cachename = model_cachename(overall_parameters, settings) # uniquely identify the run by parameters + settings
cachepath = joinpath(pwd(), "data", cachename * ".csv")
use_cache = true # change this to false if you don't want to use the cache for a run

if isfile(cachepath) && use_cache # read if possible
    println("using cache.")
    solved = CSV.read(cachepath);
else
    println("solving without cache.")
    @time solution = @suppress solve_transition(overall_parameters, settings);
    solved = solution.results;
end

filter!(row -> row.t >= 0, solved) # don't plot pre-shock values
@show ce_075_transition = 100*consumption_equivalent(solved.U[1], stationary_0.U_bar, overall_parameters);

solving without cache.
446.150888 seconds (1.46 G allocations: 669.823 GiB, 35.25% gc time)
ce_075_transition = 100 * consumption_equivalent(solved.U[1], stationary_0.U_bar, overall_parameters) = 10.802124724991934


## Zeta 1.25

### Baseline Allocations

In [37]:
sol_125 = stationary_algebraic(ζ_125, settings_defaults())
for key in (:g, :Ω, :z_hat, :λ_ii, :c)
    val = sol_125[key]
    println("$(key) = $(val)")
end

g = 0.00791340196314654
Ω = 0.5444723687853111
z_hat = 1.9813268967443167
λ_ii = 0.8937087282948921
c = 0.7854850570129506


### Counterfactual (Lowered) Allocations and Consumption Equivalent

In [38]:
sol_125_lowered = stationary_algebraic(ζ_125_lowered, settings_defaults())
for key in (:g, :Ω, :z_hat, :λ_ii, :c)
    val = sol_125_lowered[key]
    println("$(key) = $(val)")
end

g = 0.0102508227985463
Ω = 0.5167894201216091
z_hat = 1.8456817342226606
λ_ii = 0.8555713586705136
c = 0.7785010572181674


In [39]:
ce_125 = 100*consumption_equivalent(sol_125_lowered.U_bar, sol_125.U_bar, ζ_125)

11.181899562200325

### Transition Dynamics and Consumption Equivalent

In [40]:
overall_parameters = merge(ζ_125, (d_0 = d, d_T = d_lowered,));
settings = settings_defaults()
@unpack stationary_T, stationary_0, change_welfare, change_trade = compare_steady_states(overall_parameters, settings);

In [41]:
abs(stationary_T.g - sol_125_lowered.g) < 1e-5 && abs(stationary_0.g - sol_125.g) < 1e-5 # check that the grid is close enough

true

In [42]:
## commenting out as this doesn't solve well
# cachename = model_cachename(overall_parameters, settings) # uniquely identify the run by parameters + settings
# cachepath = joinpath(pwd(), "data", cachename * ".csv")
# use_cache = true # change this to false if you don't want to use the cache for a run

# if isfile(cachepath) && use_cache # read if possible
#     println("using cache.")
#     solved = CSV.read(cachepath);
# else
#     println("solving without cache.")
#     @time solution = @suppress solve_transition(overall_parameters, settings);
#     solved = solution.results;
# end

# filter!(row -> row.t >= 0, solved) # don't plot pre-shock values
# @show ce_125_transition = 100*consumption_equivalent(solved.U[1], stationary_0.U_bar, overall_parameters);